In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json


Baseline: TF-IDF + Cosine Similarity

Derin Öğrenme: Sentence-BERT (sentence-transformers/all-MiniLM-L6-v2)

Kütüphanelerimizi yükleyelim!

**1. GEREKLİ KÜTÜPHANELER**

In [2]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

**2. ARXIV VERİSİNİ STREAM EDEREK OKUMA (KAGGLE İÇİN OPTİMİZE)**

In [3]:
PATH= "/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json"

def get_data(path,max_rows=50000):
 with open(path,"r") as f:
  for i,line in enumerate(f):  #satır+index ekleyerek okur
      if i>= max_rows:
          break
      yield json.loads(line)  #belleğe almadan satır satır üretir büyük verilerde fonskiyon devam eder

data=get_data(PATH, max_rows=50000)

**3. DATAFRAME OLUŞTURMA (STREAMLIT İÇİN GEREKLİ TÜM ALANLAR)**

In [4]:
papers_data = {
    'id': [],
    'title': [],
    'abstract': [],
    'authors': [],
    'year': [],
    'categories': [],
    'link': []
}

for paper in tqdm(data):  #tqdm bir progress bardır kaç satır okunduğu görülür
    if paper.get("title") and paper.get("abstract"):  #baslıgı ve özeti olmayan makaleyi almıyoruz
     year = int(paper["update_date"].split("-")[0])  #tarihten sadece yılı alıyoruz

     papers_data["id"].append(paper["id"])
     papers_data["title"].append(paper["title"].replace("\n", " ").strip())   #strip baştaki sondaki boşlukları siler
     papers_data["abstract"].append(paper["abstract"].replace("\n", " ").strip())
     papers_data["authors"].append(paper["authors"])
     papers_data["year"].append(year)
     papers_data["categories"].append(paper["categories"])
     papers_data["link"].append(f"https://arxiv.org/abs/{paper['id']}")

50000it [00:01, 34732.62it/s]


**4. DATAFRAME + ANA KATEGORİ ÇIKARMA**

In [5]:
df=pd.DataFrame(papers_data)
df.head(10)

,id,title,abstract,authors,year,categories,link
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturbati...,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",2008,hep-ph,https://arxiv.org/abs/0704.0001
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-pe...",Ileana Streinu and Louis Theran,2008,math.CO cs.CG,https://arxiv.org/abs/0704.0002
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is describe...,Hongjun Pan,2008,physics.gen-ph,https://arxiv.org/abs/0704.0003
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle n...,David Callan,2007,math.CO,https://arxiv.org/abs/0704.0004
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\Lam...,Wael Abu-Shammala and Alberto Torchinsky,2013,math.CA math.FA,https://arxiv.org/abs/0704.0005
5,0704.0006,Bosonic characters of atomic Cooper pairs acro...,We study the two-particle wave function of pai...,Y. H. Pong and C. K. Law,2015,cond-mat.mes-hall,https://arxiv.org/abs/0704.0006
6,0704.0007,Polymer Quantum Mechanics and its Continuum Limit,A rather non-standard quantum representation o...,"Alejandro Corichi, Tatjana Vukasinac and Jose ...",2008,gr-qc,https://arxiv.org/abs/0704.0007
7,0704.0008,Numerical solution of shock and ramp compressi...,A general formulation was developed to represe...,Damian C. Swift,2009,cond-mat.mtrl-sci,https://arxiv.org/abs/0704.0008
8,0704.0009,"The Spitzer c2d Survey of Large, Nearby, Inste...",We discuss the results from the combined IRAC ...,"Paul Harvey, Bruno Merin, Tracy L. Huard, Luis...",2010,astro-ph,https://arxiv.org/abs/0704.0009
9,0704.0010,"Partial cubes: structures, characterizations, ...",Partial cubes are isometric subgraphs of hyper...,Sergei Ovchinnikov,2007,math.CO,https://arxiv.org/abs/0704.0010


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          50000 non-null  object
 1   title       50000 non-null  object
 2   abstract    50000 non-null  object
 3   authors     50000 non-null  object
 4   year        50000 non-null  int64 
 5   categories  50000 non-null  object
 6   link        50000 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.7+ MB


Ana kategorileri filtrelemeliyiz

Veri setinde her alandan belli miktarda veri almalyıız
Ana alanlardan ilk parçaları alıcaz cs.LG->cs olacak

In [7]:
df["main_category"]=df["categories"].apply(lambda x: x.split()[0].split(".")[0])

In [8]:
df["main_category"].value_counts().head(10) #en sık geçen 10 alan

main_category
math        10181
astro-ph     9208
cond-mat     8367
hep-ph       3591
physics      3231
hep-th       3020
quant-ph     2646
cs           2433
gr-qc        1670
math-ph      1047
Name: count, dtype: int64

En fazla makalesi olan ve farklı olan alanları seçelim

**5. DENGELİ VERİ SETİ (BİLİM ALANLARI)**

In [9]:
selected_categories=[
"cs","math","cond-mat","astro-ph","physics","eess"]

filtered=df[df["main_category"].isin(selected_categories)]

In [10]:
df.columns

Index(['id', 'title', 'abstract', 'authors', 'year', 'categories', 'link',
       'main_category'],
      dtype='object')

balanced_df değişkeni ana kategorileri gruplar her kategoriden 2000 veri alalım 6 kategori*2000=
12000 makalemiz olacak

In [11]:
balanced_df=(filtered
            .groupby("main_category")
            .sample(n=2000,random_state=42)
            .reset_index(drop=True)
            )

In [12]:
balanced_df["main_category"].value_counts()

main_category
astro-ph    2000
cond-mat    2000
cs          2000
math        2000
physics     2000
Name: count, dtype: int64

In [13]:
balanced_df.to_pickle("balanced_df.pkl")


6 ana alan var her birinden 2000 örneklem toplam 12000 makale var.(Veri dengesizliğine ve model doğruluğuna iyi yönde etkili)

TF-IDF + COSINE SIMILARITY
(Baseline İçerik-Tabanlı Makale Öneri Sistemi)

title+abstract birleştirmek lazım böylece TF-IDF daga anlamlı çalışır

strip(boşlukları temizler)
"." başlıkların sonuna nokta konur (cümle sınırı)
TF-IDF dokuman bazlı title+abstract konu+detay birlikte öğrenir daha doğru benzerlik

**6. METİN BİRLEŞTİRME (TF-IDF & SBERT)**

In [14]:
balanced_df["text"]=(balanced_df["title"].str.strip()+"."+
                    balanced_df["abstract"].str.strip())

**7. TF-IDF + COSINE SIMILARITY (BASELINE)**

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity #benzerlik matrisi

TF-IDF VEKTÖRLEŞTİRME

In [16]:
tfidf=TfidfVectorizer(stop_words="english",max_features=5000)

In [17]:
tfidf_matrix=tfidf.fit_transform(balanced_df["text"])

In [18]:
tfidf_matrix.shape

(10000, 5000)

Benzerlik matrisi

In [19]:
cos_sim=cosine_similarity(tfidf_matrix,tfidf_matrix)


list(enumerate):index vererek makale benzerlik skorlarını sıralar
sorted( 1 benzerlik skoruyla büyükten küçüğe(reverse=true) sıralar
iloc:index e göre satır seçer

Öneri sistemimiz için fonksiyon kuralım

In [20]:
def recommend_tfidf(paper_index, top_n=5):
    similarity_scores = list(enumerate(cos_sim[paper_index]))
    similarity_scores = sorted(similarity_scores,key=lambda x: x[1],reverse=True)
    
    top_idx = [i for i, _ in similarity_scores[1:top_n+1]] #makalenin kendisini atla en benzerlerini alalim
    
    return balanced_df.iloc[top_idx][
       ["title", "authors", "year", "main_category", "link"]
    ] #benzer makalelerin indexleriyle makaleleri getir


In [21]:
idx=100
balanced_df.iloc[idx][["title", "authors", "year", "main_category", "link"]]

title            The 2006 Radio Outburst of a Microquasar Cyg X...
authors          M. Tsuboi, T. Tosaki, N. Kuno, K. Nakanishi, T...
year                                                          2015
main_category                                             astro-ph
link                               https://arxiv.org/abs/0710.2722
Name: 100, dtype: object

In [22]:
recommend_tfidf(idx,top_n=5)

,title,authors,year,main_category,link
979,Detailed study of the GRB 030329 radio aftergl...,"A.J. van der Horst, A. Kamble, L. Resmi, R.A.M...",2009,astro-ph,https://arxiv.org/abs/0706.1321
1796,8.4GHz VLBI observations of SN2004et in NGC6946,"I. Marti-Vidal, J.M.Marcaide, A. Alberdi, J.C....",2009,astro-ph,https://arxiv.org/abs/0705.3853
638,Mapping Observations of 6.7 GHz Methanol Maser...,"Koichiro Sugiyama, Kenta Fujisawa, Akihiro Doi...",2015,astro-ph,https://arxiv.org/abs/0710.4872
73,The pre-outburst flare of the A 0535+26 August...,"I.Caballero, A.Santangelo, P.Kretschmar, R.Sta...",2009,astro-ph,https://arxiv.org/abs/0801.3167
1641,Searching for coronal radio emission from prot...,"Jan Forbrich, Maria Massi, Eduardo Ros, Andrea...",2009,astro-ph,https://arxiv.org/abs/0704.3557


Seçilen astrofizik makalesi için TF-IDF tabanlı içerik bazlı öneri sistemi, alan içi terminolojik benzerliği başarıyla yakalamış ve önerilerini aynı ana kategori (astro-ph) içerisinden üretmiştir. Bu durum, modelin teknik ve bilimsel tutarlılığını göstermektedir.

kelime bazlı benzerliği ölçtük fakat yeterli değil 

SBERT = derin öğrenme tabanlı içerik bazlı öneri sistemi

SBERT cümlelerin anlamını öğrenir alanlar arası benzerliği yakalar

In [23]:
!pip install sentence-transformers

**8. SBERT (DERİN ÖĞRENME TABANLI)**

In [24]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

2026-01-13 21:46:02.823687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768340763.087250      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768340763.169201      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768340763.819162      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768340763.819210      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768340763.819214      17 computation_placer.cc:177] computation placer alr

SBERT için 384 boyutlu hızlı bu modeli alalım

In [25]:
model=SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
texts=balanced_df["text"].tolist()

makale-> 1 embedding

In [27]:
embeddings=model.encode(texts,show_progress_bar=True)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [28]:
embeddings.shape

(10000, 384)

In [29]:
np.save("/kaggle/working/embeddings.npy", embeddings)
balanced_df.to_pickle("/kaggle/working/balanced_df.pkl")

9. SBERT BENZERLİK + ÖNERİ

In [30]:
cos_sim_sbert=cosine_similarity(embeddings,embeddings)

In [31]:
def recommend_sbert(paper_index, top_n=5):
    similarity_scores = list(enumerate(cos_sim_sbert[paper_index]))
    
    similarity_scores = sorted(
        similarity_scores,
        key=lambda x: x[1],
        reverse=True
    )
    
    top_indices = [i for i, _ in similarity_scores[1:top_n + 1]]
    
    return balanced_df.iloc[top_indices][
        ["title", "authors", "year", "main_category", "link"]
    ]


SBERT ÖNERİ TESTİ

In [32]:
idx = 9000
balanced_df.iloc[idx][
    ["title", "authors", "year", "main_category", "link"]
]

recommend_sbert(idx, top_n=5)

,title,authors,year,main_category,link
289,First real time detection of Be7 solar neutrin...,Borexino Collaboration,2012,astro-ph,https://arxiv.org/abs/0708.2251
8516,Study of Micro Pixel Photon Counters for a hig...,"N.D'Ascenzo (University of Hamburg, DESY), A. ...",2007,physics,https://arxiv.org/abs/0711.1287
9818,Detectors and flux instrumentation for future ...,"T. Abe, H. Aihara, C. Andreopoulos, A. Ankowsk...",2009,physics,https://arxiv.org/abs/0712.4129
9324,The Quest for the Ideal Scintillator for Hybri...,"B.K.Lubsandorzhiev, B.Combettes",2009,physics,https://arxiv.org/abs/0710.2069
1193,Potential for Precision Measurement of Solar N...,"Y.H. Huang, R.E. Lanou, H.J. Maris, G.M. Seide...",2009,astro-ph,https://arxiv.org/abs/0711.4095


In [33]:
balanced_df.to_pickle("/kaggle/working/balanced_df.pkl")

import numpy as np
np.save("/kaggle/working/embeddings.npy", embeddings)


Şimdi makale öneri sistemimiz için içerik tabanı bitirdik ve ekstra olarak hibrit sistem için Kullanıcı tabanı eklemeliyiz

Kullanıcıyı:

ID

Okuduğu / beğendiği makaleler

İlgi alanları 

**3.1 Kullanıcı Veri Yapısı** User profile modeling

In [34]:
import numpy as np

def generate_users(df, n_users=10, likes_per_user=5):
    """
    Alan-temelli sentetik kullanıcı profilleri üretir
    """
    users = {}
    categories = df["main_category"].unique()

    for uid in range(1, n_users + 1):
        fav_cat = np.random.choice(categories)

        liked_papers = (
            df[df["main_category"] == fav_cat]
            .sample(likes_per_user, random_state=uid)
            .index
            .tolist()
        )

        users[uid] = {
            "favorite_category": fav_cat,
            "liked_papers": liked_papers
        }

    return users


In [35]:
users = generate_users(balanced_df, n_users=10, likes_per_user=5)

Kullanıcı embedding 
sevdiği makale -> kullanıcıyı tanımlar

userEmbedding= 1/N ∑ PaperEmbedding

In [36]:
def create_user_embedding(user_id, users, embeddings):
    liked = users[user_id]["liked_papers"]

    if len(liked) == 0:
        return None

    user_emb = np.mean(embeddings[liked], axis=0)
    return user_emb


In [37]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_for_user(user_id, users, embeddings, df, top_n=10):
    user_emb = create_user_embedding(user_id, users, embeddings)

    if user_emb is None:
        return None

    scores = cosine_similarity(
        user_emb.reshape(1, -1),
        embeddings
    )[0]

    top_idx = scores.argsort()[::-1][:top_n]

    return df.iloc[top_idx][
        ["title", "authors", "year", "main_category", "link"]
    ]


In [38]:
recommend_for_user(
    user_id=1,
    users=users,
    embeddings=embeddings,
    df=balanced_df,
    top_n=5
)

,title,authors,year,main_category,link
5282,Tighter and Stable Bounds for Marcum Q-Function,Jiangping Wang,2007,cs,https://arxiv.org/abs/0712.3896
5699,On the Polyphase Decomposition for Design of G...,Massimiliano Laddomada,2016,cs,https://arxiv.org/abs/0707.2436
5876,Another view of the Gaussian algorithm,"Ali Akhavi (GREYC), C\'eline Moreira (GREYC)",2007,cs,https://arxiv.org/abs/0707.0644
5567,On the Monotonicity of the Generalized Marcum ...,"Vasilios M. Kapinas, Sotirios K. Mihos and Geo...",2010,cs,https://arxiv.org/abs/0712.4103
5210,Guarding curvilinear art galleries with vertex...,Menelaos I. Karavelas and Elias P. Tsigaridas,2009,cs,https://arxiv.org/abs/0802.2594


In [39]:
def hybrid_recommendation(
    paper_idx,
    user_id,
    users,
    embeddings,
    df,
    alpha=0.6,
    top_n=10
):
    # 1️ İçerik tabanlı skor
    content_scores = cosine_similarity(
        embeddings[paper_idx].reshape(1, -1),
        embeddings
    )[0]

    # 2️ Kullanıcı tabanlı skor
    user_emb = create_user_embedding(user_id, users, embeddings)
    user_scores = cosine_similarity(
        user_emb.reshape(1, -1),
        embeddings
    )[0]

    # 3 Hibrit skor
    final_scores = alpha * content_scores + (1 - alpha) * user_scores

    top_idx = final_scores.argsort()[::-1][1:top_n+1]

    return df.iloc[top_idx][
        ["title", "authors", "year", "main_category", "link"]
    ]


In [40]:
hybrid_recommendation(
    paper_idx=100,
    user_id=1,
    users=users,
    embeddings=embeddings,
    df=balanced_df,
    top_n=5
)


,title,authors,year,main_category,link
1187,Observations of microquasars with the MAGIC te...,"J. Rico, M. Rissi, P. Bordas, V. Bosch-Ramon, ...",2009,astro-ph,https://arxiv.org/abs/0709.2288
1813,A new explanation for the SFXTs outbursts,"L. Sidoli (1), P. Romano (2), S. Mereghetti (1...",2009,astro-ph,https://arxiv.org/abs/0712.3160
1548,Observed flux density enhancement at submillim...,"G. Cristiani, C.G. Gim\'enez de Castro, M.L. L...",2008,astro-ph,https://arxiv.org/abs/0706.0012
1661,Outer jet X-ray and radio emission in R Aquari...,"E. Kellogg (1), C. Anderson (1), K. Korreck (1...",2009,astro-ph,https://arxiv.org/abs/0705.2570
73,The pre-outburst flare of the A 0535+26 August...,"I.Caballero, A.Santangelo, P.Kretschmar, R.Sta...",2009,astro-ph,https://arxiv.org/abs/0801.3167
